## Update dataset for cvd mortality, use both ed_visits and inpatient data

In [118]:
import pandas as pd
import os
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import math

datatype = "quality"

In [119]:
if datatype == "train": 
    in_ad = pd.read_csv("/home/bhatti/dataset/VCHAMPS/inpatient_admissions_train.csv", index_col=0)
    ed = pd.read_csv("/home/bhatti/dataset/VCHAMPS/ed_visits_train.csv", index_col=0)
    death = pd.read_csv("/home/bhatti/dataset/VCHAMPS/death_train.csv", index_col=0)
if datatype == "test": 
    in_ad = pd.read_csv("/data/public/MLA/VCHAMPS-Test/inpatient_admissions_test.csv", index_col=0)
    ed = pd.read_csv("/data/public/MLA/VCHAMPS-Test/ed_visits_test.csv", index_col=0)
    death = pd.read_csv("/data/public/MLA/VCHAMPS-Test/death_test.csv", index_col=0)
if datatype == "quality": 
    in_ad = pd.read_csv("/data/public/MLA/VCHAMPS-Quality/inpatient_admissions_qual_Formatfixed.csv", index_col=0)
    ed = pd.read_csv("/data/public/MLA/VCHAMPS-Quality/ed_visits_qual_Formatfixed.csv", index_col=0)
    in_ad.rename(columns = {'Age.at.admission': "Age at admission", "Admission.date": "Admission date", "Discharge.date": "Discharge date",
                            "Admitting.unit.service": "Admitting unit service", "Discharging.unit.service": "Discharging unit service",
                            "Admitting.specialty": "Admitting specialty", "Discharging.specialty": "Discharging specialty", 
                            "First.listed.discharge.diagnosis.icd10.subcategory": "First listed discharge diagnosis icd10 subcategory",
                            "Second.listed.discharge.diagnosis.icd10.subcategory": "Second listed discharge diagnosis icd10 subcategory",
                            "Discharge.disposition": "Discharge disposition", "Died.during.admission": "Died during admission"}, inplace = True)
    ed.rename(columns = {'Age.at.ed.visit': "Age at ed visit", 'Ed.visit.start.date': 'Ed visit start date', 
                         'Discharge.date.ed': 'Discharge date ed', 'Died.during.ed.visit': 'Died during ed visit',
                         'First.listed.diagnosis.icd10.subcategory': 'First listed diagnosis icd10 subcategory',
                         'Second.listed.diagnosis.icd10.subcategory': 'Second listed diagnosis icd10 subcategory'}, inplace = True)
    

In [120]:
ed.columns

Index(['X', 'Internalpatientid', 'Age at ed visit', 'Ed visit start date',
       'Discharge date ed', 'Died during ed visit',
       'First listed diagnosis icd10 subcategory',
       'Second listed diagnosis icd10 subcategory', 'State'],
      dtype='object')

In [121]:
len(set(in_ad["Internalpatientid"]) & set(ed["Internalpatientid"]))

392

In [122]:
in_ad.columns

Index(['X', 'Internalpatientid', 'Age at admission', 'Admission date',
       'Discharge date', 'Admitting unit service', 'Discharging unit service',
       'Admitting specialty', 'Discharging specialty',
       'First listed discharge diagnosis icd10 subcategory',
       'Second listed discharge diagnosis icd10 subcategory',
       'Discharge disposition', 'Died during admission',
       'Outpatientreferralflag', 'Serviceconnectedflag', 'Agentorangeflag',
       'State'],
      dtype='object')

In [123]:
ed = ed.rename(columns={'Age at ed visit': 'Age at admission', 'Ed visit start date': 'Admission date', 
'Discharge date ed': 'Discharge date', 'Died during ed visit': 'Died during admission',
'First listed diagnosis icd10 subcategory': 'First listed discharge diagnosis icd10 subcategory',
'Second listed diagnosis icd10 subcategory': 'Second listed discharge diagnosis icd10 subcategory'})

In [124]:
ed.columns

Index(['X', 'Internalpatientid', 'Age at admission', 'Admission date',
       'Discharge date', 'Died during admission',
       'First listed discharge diagnosis icd10 subcategory',
       'Second listed discharge diagnosis icd10 subcategory', 'State'],
      dtype='object')

In [125]:
full = pd.concat([ed, in_ad[['Internalpatientid', 'Age at admission', 'Admission date',
       'Discharge date', 'Died during admission',
       'First listed discharge diagnosis icd10 subcategory',
       'Second listed discharge diagnosis icd10 subcategory', 'State']]])

In [126]:
full.columns

Index(['X', 'Internalpatientid', 'Age at admission', 'Admission date',
       'Discharge date', 'Died during admission',
       'First listed discharge diagnosis icd10 subcategory',
       'Second listed discharge diagnosis icd10 subcategory', 'State'],
      dtype='object')

In [127]:
full

,X,Internalpatientid,Age at admission,Admission date,Discharge date,Died during admission,First listed discharge diagnosis icd10 subcategory,Second listed discharge diagnosis icd10 subcategory,State
1,16.0,106440,64.954978,2021-01-16 01:04:13.0,2021-01-16 04:50:26.0,NaN,Pressure ulcer of heel,"Chronic kidney disease, unspecified",Texas
2,119.0,129977,82.745838,2018-02-02 14:29:29.0,2018-02-02 21:16:32.0,NaN,"Sepsis, unspecified organism","Urinary tract infection, site not specified",Oklahoma
3,172.0,106440,63.700552,2019-10-15 13:14:25.0,2019-10-15 20:19:09.0,NaN,Cellulitis and acute lymphangitis of other par...,"Hematuria, unspecified",Texas
4,338.0,131548,81.817275,2020-06-22 05:08:00.0,2020-06-22 09:07:39.0,NaN,Abdominal distension (gaseous),Unspecified abdominal pain,Illinois
5,355.0,129977,84.977907,2020-04-28 10:11:58.0,2020-04-28 16:08:41.0,NaN,Constipation,"Urinary tract infection, site not specified",Oklahoma
...,...,...,...,...,...,...,...,...,...
4006,NaN,99870,87.481429,2008-12-07 11:05:40.0,2008-12-27 10:57:14.0,NaN,Spinal stenosis,"Other thoracic, thoracolumbar and lumbosacral ...",South Dakota
4007,NaN,95448,47.736311,2009-03-21 14:55:07.0,2009-03-23 12:33:43.0,NaN,Other chest pain,Hypertensive chronic kidney disease with stage...,Nevada
4008,NaN,98416,78.000151,2005-09-12 01:19:43.0,2005-09-18 19:18:02.0,NaN,"Heart failure, unspecified","Nonrheumatic aortic valve disorder, unspecified",California
4009,NaN,99137,88.265051,2019-01-13 21:01:36.0,2019-01-20 22:03:19.0,NaN,Diastolic (congestive) heart failure,"Chronic kidney disease, stage 4 (severe)",Texas


In [128]:
for i in full.columns:
    print(i, full[full[i].isna()].shape[0]/len(full))

X 0.47506219642222486
Internalpatientid 0.0
Age at admission 0.0
Admission date 0.0
Discharge date 0.0005923468783319512
Died during admission 1.0
First listed discharge diagnosis icd10 subcategory 0.0
Second listed discharge diagnosis icd10 subcategory 0.0
State 0.0


#### Died

In [129]:
full["Died during admission"] = full["Died during admission"].replace({"Yes":1, "No": 0})

#### Stay length

In [130]:
full["Admission date"] = pd.to_datetime(full["Admission date"], format='%Y-%m-%d %H:%M:%S.%f')
full["Discharge date"] = pd.to_datetime(full["Discharge date"], format='%Y-%m-%d %H:%M:%S.%f')
full["stay_length"] = (full["Discharge date"] - full["Admission date"]).dt.days + round((full["Discharge date"] - full["Admission date"]).dt.seconds/3600/24,2)

In [131]:
full["stay_length"] = full["stay_length"].fillna(full["stay_length"].mean())

### Age at admission

In [132]:
print(full["Age at admission"].min())
print(full["Age at admission"].max())

27.61847039
104.0575511


In [133]:
full['Age 20-40'] = [0] * len(full)
full['Age 40-60'] = [0] * len(full)
full['Age 60-80'] = [0] * len(full)
full['Age 80-100'] = [0] * len(full)
full['Age 100-120'] = [0] * len(full)
def age_category(data):
    data.loc[(data['Age at admission'] > 20) & (data['Age at admission'] <= 40), 'Age 20-40'] = 1
    data.loc[(data['Age at admission'] > 40) & (data['Age at admission'] <= 60), 'Age 40-60'] = 1
    data.loc[(data['Age at admission'] > 60) & (data['Age at admission'] <= 80), 'Age 60-80'] = 1
    data.loc[(data['Age at admission'] > 80) & (data['Age at admission'] <= 100), 'Age 80-100'] = 1
    data.loc[(data['Age at admission'] > 100) & (data['Age at admission'] <= 120), 'Age 100-120'] = 1
    return data
full = age_category(full)
full

,X,Internalpatientid,Age at admission,Admission date,Discharge date,Died during admission,First listed discharge diagnosis icd10 subcategory,Second listed discharge diagnosis icd10 subcategory,State,stay_length,Age 20-40,Age 40-60,Age 60-80,Age 80-100,Age 100-120
1,16.0,106440,64.954978,2021-01-16 01:04:13,2021-01-16 04:50:26,NaN,Pressure ulcer of heel,"Chronic kidney disease, unspecified",Texas,0.16,0,0,1,0,0
2,119.0,129977,82.745838,2018-02-02 14:29:29,2018-02-02 21:16:32,NaN,"Sepsis, unspecified organism","Urinary tract infection, site not specified",Oklahoma,0.28,0,0,0,1,0
3,172.0,106440,63.700552,2019-10-15 13:14:25,2019-10-15 20:19:09,NaN,Cellulitis and acute lymphangitis of other par...,"Hematuria, unspecified",Texas,0.29,0,0,1,0,0
4,338.0,131548,81.817275,2020-06-22 05:08:00,2020-06-22 09:07:39,NaN,Abdominal distension (gaseous),Unspecified abdominal pain,Illinois,0.17,0,0,0,1,0
5,355.0,129977,84.977907,2020-04-28 10:11:58,2020-04-28 16:08:41,NaN,Constipation,"Urinary tract infection, site not specified",Oklahoma,0.25,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4006,NaN,99870,87.481429,2008-12-07 11:05:40,2008-12-27 10:57:14,NaN,Spinal stenosis,"Other thoracic, thoracolumbar and lumbosacral ...",South Dakota,19.99,0,0,0,1,0
4007,NaN,95448,47.736311,2009-03-21 14:55:07,2009-03-23 12:33:43,NaN,Other chest pain,Hypertensive chronic kidney disease with stage...,Nevada,1.90,0,1,0,0,0
4008,NaN,98416,78.000151,2005-09-12 01:19:43,2005-09-18 19:18:02,NaN,"Heart failure, unspecified","Nonrheumatic aortic valve disorder, unspecified",California,6.75,0,0,1,0,0
4009,NaN,99137,88.265051,2019-01-13 21:01:36,2019-01-20 22:03:19,NaN,Diastolic (congestive) heart failure,"Chronic kidney disease, stage 4 (severe)",Texas,7.04,0,0,0,1,0


#### CVD

In [134]:
cardiovascular = ["Heart failure, unspecified", "Other heart failure",
                  "Cardiogenic shock", "Hypertensive heart disease with heart failure",
                  "Hypertensive heart and kidney disease with heart failure", "Unstable angina", "Other forms of chronic ischemic heart disease",
                  "Atherosclerotic heart disease of native coronary artery", "Atrial fibrillation", "Atrial flutter",
                  "Supraventricular tachycardia", "Ventricular tachycardia"]

In [135]:
full["cd_diagnosis"] = [0] * len(full)
def cd_diagnosis(data):
    data.loc[(full["Second listed discharge diagnosis icd10 subcategory"].str.contains('|'.join(cardiovascular))
      | full["Second listed discharge diagnosis icd10 subcategory"].str.contains("Systolic (congestive) heart failure", regex=False)
      | full["Second listed discharge diagnosis icd10 subcategory"].str.contains("Diastolic (congestive) heart failure", regex=False)
      | full["Second listed discharge diagnosis icd10 subcategory"].str.contains("Combined systolic (congestive) and diastolic (congestive) heart failure", regex=False)
      | full["Second listed discharge diagnosis icd10 subcategory"].str.contains("ST elevation (STEMI) myocardial infarction", regex=False)
      | full["Second listed discharge diagnosis icd10 subcategory"].str.contains("Non-ST elevation (NSTEMI) myocardial infarction", regex=False)
      | full["First listed discharge diagnosis icd10 subcategory"].str.contains('|'.join(cardiovascular))
      | full["First listed discharge diagnosis icd10 subcategory"].str.contains("Systolic (congestive) heart failure", regex=False)
      | full["First listed discharge diagnosis icd10 subcategory"].str.contains("Diastolic (congestive) heart failure", regex=False)
      | full["First listed discharge diagnosis icd10 subcategory"].str.contains("Combined systolic (congestive) and diastolic (congestive) heart failure", regex=False)
      | full["First listed discharge diagnosis icd10 subcategory"].str.contains("ST elevation (STEMI) myocardial infarction", regex=False)
      | full["First listed discharge diagnosis icd10 subcategory"].str.contains("Non-ST elevation (NSTEMI) myocardial infarction", regex=False)), "cd_diagnosis"] = 1
    return data
full = cd_diagnosis(full)
full


,X,Internalpatientid,Age at admission,Admission date,Discharge date,Died during admission,First listed discharge diagnosis icd10 subcategory,Second listed discharge diagnosis icd10 subcategory,State,stay_length,Age 20-40,Age 40-60,Age 60-80,Age 80-100,Age 100-120,cd_diagnosis
1,16.0,106440,64.954978,2021-01-16 01:04:13,2021-01-16 04:50:26,NaN,Pressure ulcer of heel,"Chronic kidney disease, unspecified",Texas,0.16,0,0,1,0,0,0
2,119.0,129977,82.745838,2018-02-02 14:29:29,2018-02-02 21:16:32,NaN,"Sepsis, unspecified organism","Urinary tract infection, site not specified",Oklahoma,0.28,0,0,0,1,0,0
3,172.0,106440,63.700552,2019-10-15 13:14:25,2019-10-15 20:19:09,NaN,Cellulitis and acute lymphangitis of other par...,"Hematuria, unspecified",Texas,0.29,0,0,1,0,0,0
4,338.0,131548,81.817275,2020-06-22 05:08:00,2020-06-22 09:07:39,NaN,Abdominal distension (gaseous),Unspecified abdominal pain,Illinois,0.17,0,0,0,1,0,0
5,355.0,129977,84.977907,2020-04-28 10:11:58,2020-04-28 16:08:41,NaN,Constipation,"Urinary tract infection, site not specified",Oklahoma,0.25,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4006,NaN,99870,87.481429,2008-12-07 11:05:40,2008-12-27 10:57:14,NaN,Spinal stenosis,"Other thoracic, thoracolumbar and lumbosacral ...",South Dakota,19.99,0,0,0,1,0,0
4007,NaN,95448,47.736311,2009-03-21 14:55:07,2009-03-23 12:33:43,NaN,Other chest pain,Hypertensive chronic kidney disease with stage...,Nevada,1.90,0,1,0,0,0,0
4008,NaN,98416,78.000151,2005-09-12 01:19:43,2005-09-18 19:18:02,NaN,"Heart failure, unspecified","Nonrheumatic aortic valve disorder, unspecified",California,6.75,0,0,1,0,0,1
4009,NaN,99137,88.265051,2019-01-13 21:01:36,2019-01-20 22:03:19,NaN,Diastolic (congestive) heart failure,"Chronic kidney disease, stage 4 (severe)",Texas,7.04,0,0,0,1,0,1


### Final dataset

In [136]:
def final_set_before(ids, group):
    
    readmission = 0
    num_admissions = group["Age at admission"].nunique()
    if num_admissions > 1: readmission=1

    age_mean = group["Age at admission"].mean()
    age_std = group["Age at admission"].std()
    if group["Age at admission"].nunique() == 1: age_std = 0

    min_age = group["Age at admission"].min()
    max_age = group["Age at admission"].max()

    freq = num_admissions/(math.floor(max_age - min_age) + 1)

    min_stay = group["stay_length"].min()
    max_stay = group["stay_length"].min()
    stay_mean = group["stay_length"].mean()
    stay_std = group["stay_length"].std()
    if group["stay_length"].nunique() == 1: stay_std = 0

    # num_cvd_readmission = max(0, group['cd_diagnosis'].sum() - 1)
    num_cvd_admission = group['cd_diagnosis'].sum()

    cvd = 0
    if group["cd_diagnosis"].sum() > 0: cvd = 1

    if (group.iloc[len(group)-1]["Died during admission"] == 1) and (group.iloc[len(group)-1]['cd_diagnosis'] == 1): died_by_cvd = 1
    else: died_by_cvd = 0

    df = pd.DataFrame(data={'Internalpatientid': [ids], 'num_stays': [num_admissions], 'stay_length': group["stay_length"].sum(), 
                            "num_cvd_admission": [num_cvd_admission], "CVD": [cvd],
                            'Age 20-40': group["Age 20-40"].sum(), 'Age 40-60': group["Age 40-60"].sum(), 'Age 60-80':group["Age 60-80"].sum(), 
                            'Age 80-100':group["Age 80-100"].sum(), 'Age 100-120':group["Age 100-120"].sum(), "age_mean": [age_mean], "age_std": [age_std], 
                            "age_min": [min_age], "age_max": [max_age], "stay_min": [min_stay], "stay_max": [max_stay], "stay_mean": [stay_mean],
                            "stay_std": [stay_std], "freq": [round(freq,2)], "died_by_cvd": died_by_cvd
                            })
    df = df.reset_index(drop=True)
    return df

In [137]:
tidy_dataset = []
for ids, group in tqdm(full.groupby("Internalpatientid")):
    group = group.sort_values(by = ['Admission date'],ascending=True).reset_index(drop = True)
    df = final_set_before(ids, group)
    tidy_dataset.append(df)
    
tidy_dataset = pd.concat(tidy_dataset)

  0%|          | 0/674 [00:00<?, ?it/s]

In [138]:
if datatype == "train": inpatient_before = pd.read_csv("/home/hassan/lily/MLA/FDA/inpatient_cvd_mortality.csv", index_col = 0)
if datatype == "test": inpatient_before = pd.read_csv("/home/hassan/lily/MLA/FDA/inpatient_cvd_mortality_test.csv", index_col = 0)
if datatype == "quality": inpatient_before = pd.read_csv("/home/hassan/lily/MLA/FDA/inpatient_cvd_mortality_quality.csv", index_col = 0)
print(inpatient_before.columns)

if datatype == "quality": 
       inpatient_before = inpatient_before[['Internalpatientid', 
       'num_unique_units', 'num_transfers',
       'unique_admitting_specialty', 'unique_discharging_specialty',
       'DOMICILIARY', 'MEDICINE', 'NHCU', 'NON-COUNT', 'OTHERS', 'PSYCHIATRY',
       'SURGERY', 'Medical', 'Mental',
       'Others_Specialty', 'Rehab', 'Gerontology']]
else: 
       inpatient_before = inpatient_before[['Internalpatientid', 
       'num_unique_units', 'num_transfers',
       'unique_admitting_specialty', 'unique_discharging_specialty',
       'DOMICILIARY', 'MEDICINE', 'NHCU', 'NON-COUNT', 'OTHERS', 'PSYCHIATRY',
       'SURGERY', 'Medical', 'Mental',
       'Others_Specialty', 'Rehab', 'Gerontology', 'Age at death']]

Index(['Internalpatientid', 'num_stays', 'stay_length', 'num_unique_units',
       'num_transfers', 'num_cvd_admission', 'CVD',
       'unique_admitting_specialty', 'unique_discharging_specialty',
       'DOMICILIARY', 'MEDICINE', 'NHCU', 'NON-COUNT', 'OTHERS', 'PSYCHIATRY',
       'SURGERY', 'Age 20-40', 'Age 40-60', 'Age 60-80', 'Age 80-100',
       'Age 100-120', 'age_mean', 'age_std', 'age_min', 'age_max', 'stay_min',
       'stay_max', 'stay_mean', 'stay_std', 'freq', 'Medical', 'Mental',
       'Others_Specialty', 'Rehab', 'Gerontology', 'died_by_cvd'],
      dtype='object')


In [139]:
if datatype != "quality":
    tidy_dataset = tidy_dataset.merge(right = death, how="left", on="Internalpatientid")
    tidy_dataset = tidy_dataset[~tidy_dataset["Age at death"].isna()]

In [140]:
tidy_dataset = tidy_dataset.merge(right = inpatient_before, how="left", on="Internalpatientid")

In [141]:
for i in tidy_dataset.columns:
    if tidy_dataset[tidy_dataset[i].isna()].shape[0] != 0: 
        print(i, tidy_dataset[tidy_dataset[i].isna()].shape[0]/len(tidy_dataset))
        tidy_dataset[i] = tidy_dataset[i].fillna(tidy_dataset[i].mean())

num_unique_units 0.06231454005934718
num_transfers 0.06231454005934718
unique_admitting_specialty 0.06231454005934718
unique_discharging_specialty 0.06231454005934718
DOMICILIARY 0.06231454005934718
MEDICINE 0.06231454005934718
NHCU 0.06231454005934718
NON-COUNT 0.06231454005934718
OTHERS 0.06231454005934718
PSYCHIATRY 0.06231454005934718
SURGERY 0.06231454005934718
Medical 0.06231454005934718
Mental 0.06231454005934718
Others_Specialty 0.06231454005934718
Rehab 0.06231454005934718
Gerontology 0.06231454005934718


In [142]:
if datatype != "quality": tidy_dataset = tidy_dataset.drop(columns=["Age at death_y"])

In [143]:
if datatype == "train": tidy_dataset.to_csv('/home/hassan/lily/MLA/FDA/inpatient_cvd_mortality.csv')
if datatype == "test": tidy_dataset.to_csv('/home/hassan/lily/MLA/FDA/inpatient_cvd_mortality_test.csv')
if datatype == "quality": tidy_dataset.to_csv('/home/hassan/lily/MLA/FDA/inpatient_cvd_mortality_quality.csv')